# Deploying a MedNIST Classifier App with MONAI Deploy App SDK (Prebuilt Model)

This tutorial demos the process of packaging up a trained model using MONAI Deploy App SDK very much like the tutorial MedNIST notebook, however this will use the `BundleOperator` class to simplify the process of defining inference for the network. This relies on the network being packaged with the application as a MONAI bundle, which is a saved Torchscript model (or just network weights) packaged with data files containing meta information describing what the network does, how to use it, and other information.

## Install SDK and MONAI

In [ ]:
!pip install --upgrade monai-deploy-app-sdk
!pip install monai Pillow  # for MONAI transforms and Pillow

## Download/Extract mednist_classifier_data.zip from Google Drive

In [ ]:
# Download mednist_classifier_data.zip
!pip install gdown 
!gdown "https://drive.google.com/uc?id=1yJ4P-xMNEfN6lIOq_u6x1eMAq1_MJu-E"

In [ ]:
# After downloading mednist_classifier_data.zip from the web browser or using gdown,
!unzip -o "mednist_classifier_data.zip"

## Create Bundle From Torchscript Object

Bundles are individual networks packaged together with metadata and configuration data. This is provided here in full with model weights extracted from the saved Torchscript network.

First thing is to create the directory structure for the bundle:

In [ ]:
!mkdir -p mednist_classifier
!mkdir -p mednist_classifier/configs
!mkdir -p mednist_classifier/models

Next is to load the Torchscript object and save its weights to the file in the bundle, this is needed here because the prebuilt model is distributed in this format:

In [ ]:
import torch
obj=torch.jit.load("classifier.zip")
state=obj.state_dict()
torch.save({k:v.clone().cpu() for k,v in state.items()},"mednist_classifier/models/model.pt")

Bundles represent their metadata as JSON or YAML files containing dictionaries of important information. One file which must always be present is `metadata.json` containing a wide range of information about the model which is both used by software and is human-readable:

In [ ]:
%%writefile mednist_classifier/configs/metadata.json

{
    "schema": "https://github.com/Project-MONAI/MONAI-extra-test-data/releases/download/0.8.1/meta_schema_20220324.json",
    "version": "0.1.0",
    "changelog": { "0.0.1": "initialize the model package structure"},
    "monai_version": "0.8.0",
    "pytorch_version": "1.10.0",
    "numpy_version": "1.21.2",
    "optional_packages_version": {    },
    "network_def": {
        "_target_": "DenseNet121",
        "spatial_dims": 2,
        "in_channels": 1,
        "out_channels": 6
    },
    "task": "MedNIST Classification",
    "description": "A pre-trained model for classifying MedNIST images",
    "authors": "MONAI team",
    "copyright": "Copyright (c) MONAI Consortium",
    "data_source": "MedNIST data from MONAI Tutorials",
    "data_type": "jpeg",
    "image_classes": "single channel data, intensity scaled to [0, 1]",
    "label_classes": "single channel data, 0 is AbdomenCT, 1 is BreastMRI, 2 is CXR, 3 is ChestCT, 4 is Hand, 5 is HeadCT",
    "pred_classes": "6 channel one-hot data",
    "intended_use": "This is an example, not to be used for diagnostic purposes",
    "network_data_format": {
        "inputs": {
            "image": {
                "type": "image",
                "format": "magnitude",
                "num_channels": 1,
                "spatial_shape": [64, 64],
                "dtype": "float32",
                "value_range": [0, 1],
                "is_patch_data": false,
                "channel_def": {"0": "image"}
            }
        },
        "outputs": {
            "pred": {
                "type": "probabilities",
                "format": "labels",
                "num_channels": 6,
                "spatial_shape": [1],
                "dtype": "float32",
                "value_range": [],
                "is_patch_data": false,
                "channel_def": {
                    "0": "AbdomenCT",
                    "1": "BreastMRI",
                    "2": "CXR",
                    "3": "ChestCT",
                    "4": "Hand",
                    "5": "HeadCT"
                }
            }
        }
    }
}

A script used to define an inference program independent of any script is provided here. Parts of this script will be used in the operator, specifically those parts defining transform sequences, devices, inferers, and the network itself:

In [5]:
%%writefile mednist_classifier/configs/inference.json

{
    "imports": [
        "$import glob",
        "$import os"
    ],
    "device": "$torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')",
    "ckpt_path": "./mednist_classifier/models/model.pt",
    "dataset_dir": "/workspace/data",
    "datalist": "$list(sorted(glob.glob(@dataset_dir + '/*/00979*.jpeg')))",
    "network_def": {
        "_target_": "DenseNet121",
        "spatial_dims": 2,
        "in_channels": 1,
        "out_channels": 6
    },
    "network": "$@network_def.to(@device)",
    "preprocessing": {
        "_target_": "Compose",
        "transforms": [
            {
                "_target_": "LoadImaged",
                "keys": "image"
            },
            {
                "_target_": "AddChanneld",
                "keys": "image"
            },
            {
                "_target_": "ScaleIntensityd",
                "keys": "image"
            },
            {
                "_target_": "EnsureTyped",
                "keys": "image",
                "device": "@device"
            }
        ]
    },
    "dataset": {
        "_target_": "Dataset",
        "data": "$[{'image': i} for i in @datalist]",
        "transform": "@preprocessing"
    },
    "dataloader": {
        "_target_": "DataLoader",
        "dataset": "@dataset",
        "batch_size": 1,
        "shuffle": false,
        "num_workers": 0
    },
    "inferer": {
        "_target_": "SimpleInferer"
    },
    "postprocessing": {
        "_target_": "Compose",
        "transforms": [
            {
                "_target_": "Activationsd",
                "keys": "pred",
                "softmax": true
            },
            {
                "_target_": "AsDiscreted",
                "keys": "pred",
                "argmax": true
            }
        ]
    },
    "handlers": [
        {
            "_target_": "CheckpointLoader",
            "_disabled_": "$not os.path.exists(@ckpt_path)",
            "load_path": "@ckpt_path",
            "load_dict": {"model": "@network"}
        }
    ],
    "evaluator": {
        "_target_": "SupervisedEvaluator",
        "device": "@device",
        "val_data_loader": "@dataloader",
        "network": "@network",
        "inferer": "@inferer",
        "postprocessing": "@postprocessing",
        "val_handlers": "@handlers",
        "amp": true
    }
}

Overwriting mednist_classifier/configs/inference.json


A new bundle Torchscript object can then be created which will contain these components into the zip file stored in the MAP created later:

In [ ]:
!python  -m monai.bundle ckpt_export network_def \
    --filepath mednist_classifier.ts \
    --ckpt_file mednist_classifier/models/model.pt \
    --meta_file mednist_classifier/configs/metadata.json \
    --config_file mednist_classifier/configs/inference.json

The app file created here is much simpler than one which doesn't use bundles because the task of defining inference can be automatically configured within the `BundleOperator`:

In [2]:
%%writefile mednist_classifier_monaideploy_bundle.py

# Copyright 2021 MONAI Consortium
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#     http://www.apache.org/licenses/LICENSE-2.0
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

import monai.deploy.core as md  # 'md' stands for MONAI Deploy (or can use 'core' instead)
from monai.deploy.core import (
    Application,
    DataPath,
    ExecutionContext,
    Image,
    InputContext,
    IOType,
    Operator,
    OutputContext,
)
from monai.transforms import AddChannel, Compose, EnsureType, ScaleIntensity

MEDNIST_CLASSES = ["AbdomenCT", "BreastMRI", "CXR", "ChestCT", "Hand", "HeadCT"]


from monai.deploy.operators import create_bundle_operator


@md.input("image", DataPath, IOType.DISK)
@md.output("image", Image, IOType.IN_MEMORY)
@md.env(pip_packages=["pillow"])
class LoadPILOperator(Operator):
    """Load image from the given input (DataPath) and set numpy array to the output (Image)."""

    def compute(self, op_input: InputContext, op_output: OutputContext, context: ExecutionContext):
        import numpy as np
        from PIL import Image as PILImage

        input_path = op_input.get().path
        if input_path.is_dir():
            input_path = next(input_path.glob("*.*"))  # take the first file

        image = PILImage.open(input_path)
        image = image.convert("L")  # convert to greyscale image
        image_arr = np.asarray(image)

        output_image = Image(image_arr)  # create Image domain object with a numpy array
        op_output.set(output_image)
        
        
@md.input("pred",dict, IOType.IN_MEMORY)
class PrintOperator(Operator):
    def compute(self, op_input: InputContext, op_output: OutputContext, context: ExecutionContext):
        print("Prediction output:",op_input.get())
        

@md.resource(cpu=1, gpu=1, memory="1Gi")
class App(Application):
    """Application class for the MedNIST classifier."""

    def compose(self):
        load_pil_op = LoadPILOperator()
        
        classifier_op = create_bundle_operator(self._context.model_path, "inference",out_type=IOType.IN_MEMORY)
        
        print_op = PrintOperator()

        self.add_flow(load_pil_op, classifier_op)
        self.add_flow(classifier_op, print_op)


if __name__ == "__main__":
    App(do_run=True)

Overwriting mednist_classifier_monaideploy_bundle.py


## Test Running App

In [3]:
!python mednist_classifier_monaideploy_bundle.py -i input/AbdomenCT_007000.jpeg -o output -m mednist_classifier.ts

/home/localek10/miniconda3/envs/deploy/lib/python3.9/site-packages/torch/cuda/__init__.py:82: UserWarning: CUDA initialization: Unexpected error from cudaGetDeviceCount(). Did you run some cuda functions before calling NumCudaDevices() that might have already set an error? Error 804: forward compatibility was attempted on non supported HW (Triggered internally at  ../c10/cuda/CUDAFunctions.cpp:112.)
  return torch._C._cuda_getDeviceCount() > 0
Going to initiate execution of operator LoadPILOperator
Executing operator LoadPILOperator (Process ID: 872887, Operator ID: 88674d0b-d6c5-4ee7-b720-eefd1bf0fe4b)
Done performing execution of operator LoadPILOperator

Going to initiate execution of operator BundleOperator
Executing operator BundleOperator (Process ID: 872887, Operator ID: 5ed226b4-6dad-4e92-89c4-6de44643a73b)
Done performing execution of operator BundleOperator

Going to initiate execution of operator PrintOperator
Executing operator PrintOperator (Process ID: 872887, Operator ID

## Package app (creating MAP Docker image)

This assumes that nvidia docker is installed in the local machine.

Please see https://docs.nvidia.com/datacenter/cloud-native/container-toolkit/install-guide.html#docker to install nvidia-docker2.

Use `-l DEBUG` option to see progress.

In [1]:
!monai-deploy package mednist_classifier_monaideploy_bundle.py  -n\
    --tag mednist_app:latest \
    --model mednist_classifier.ts

Building MONAI Application Package... Done
[2022-05-19 13:35:24,262] [INFO] (app_packager) - Successfully built mednist_app:latest


## Run the app with docker image and input file locally

In [2]:
!monai-deploy run -l DEBUG mednist_app:latest "input" "output"

Checking dependencies...
--> Verifying if "docker" is installed...

--> Verifying if "mednist_app:latest" is available...

Checking for MAP "mednist_app:latest" locally
"mednist_app:latest" found.

Reading MONAI App Package manifest...
-------------------application manifest-------------------
{
    "api-version": "0.1.0",
    "command": "python3 -u /opt/monai/app/mednist_classifier_monaideploy_bundle.py",
    "environment": {},
    "input": {
        "formats": [],
        "path": "input"
    },
    "output": {
        "format": {},
        "path": "output"
    },
    "sdk-version": "0.3.0+7.g1aa4d10.dirty",
    "timeout": 0,
    "version": "0.0.0",
    "working-directory": "/var/monai/"
}
----------------------------------------------

-------------------package manifest-------------------
{
    "api-version": "0.1.0",
    "application-root": "/var/monai/",
    "models": [
        {
            "name": "mednist_classifier-75dac13a25625be77add180d23738cd9b2dc9f08b512e5df3291324999dee6